# Question 2 <a class="tocSkip"></a>
Consider the linear system of equations

$$
\begin{pmatrix}
  2 & 1 & 1 & 1 \\
  1 & 2 & 1 & 1 \\
  1 & 1 & 3 & 1 \\
  1 & 1 & 1 & 3 \\
\end{pmatrix}
\begin{pmatrix}
  x_1 \\ x_2 \\ x_3 \\ x_4
\end{pmatrix}
=
\begin{pmatrix}
  1 \\ 1 \\ 0 \\ 0
\end{pmatrix}
$$

**2.1** Using an initial guess of ${\bf x}^0 = (0,0,0,0)$, compute the solution using the Conjugate Gradient algorithm. You may use any code from the lecture notes, but don't use any external implementation like `scipy`'s. You do not need to do any calculations by hand.You do need to show (or print) **every** iteration vector ${\bf x}^i$ in your answer. How many iterations were required to reach the exact solution?

## Solution <a class="tocSkip"></a>

In [4]:
import numpy as np
from pprint import pprint

In [5]:
A = np.array([[2,1,1,1],[1,2,1,1],[1,1,3,1],[1,1,1,3]])
b = np.array([1,1,0,0])
x0 = np.array([0,0,0,0])

In the first iteration of the Conjugate Gradient method, we compute the initial residual, and alpha to arrive at the minimum along the line spanned by that initial residual vector:

In [6]:
r0 = b - A @ x0
alpha0 = np.dot(r0, r0) / np.dot(r0, A @ r0)
x1 = x0 + alpha0 * r0
pprint(x1)

array([0.33333333, 0.33333333, 0.        , 0.        ])


We compute the residual again, but in the second iteration we need a direction ${\bf p}^{(1)}$ that's $A$-orthogonal to ${\bf r}^{(0)}$:

In [7]:
r1 = b - A @ x1
beta = np.dot(r1, r1)/np.dot(r0, r0)
# or alternatively beta = -np.dot(r0, A @ r1)/np.dot(r0, A @ r0)
p1 = r1 + beta * r0

And again we aim to arrive in the minimum along the line spanned by ${\bf p}^{(1)}$

In [8]:
alpha1 = np.dot(p1, r1) / np.dot(p1, A @ p1)
# or alternatively: alpha1 = np.dot(r1, r1) / np.dot(p1, A @ p1)
x2 = x1 + alpha1 * p1
pprint(x2)

array([ 0.5 ,  0.5 , -0.25, -0.25])


Now if we compute residual again:

In [9]:
r2 = b - A @ x2
print(r2)

[0. 0. 0. 0.]


we have obtained a zero residual, which means that we have exactly solved $\underline{\bf A}{\bf x} = {\bf b}$ with 
$${\bf x}={\bf x}^{(2)} = \begin{pmatrix} \tfrac 12 \\ \tfrac 12 \\ -\tfrac 14 \\ -\tfrac 14 \end{pmatrix}$$

in **two** iterations.

### General feedback <a class="tocSkip"></a>
A number of students have simply taken the full conjugate gradient algorithm and put in a number of print statements. This is absolutely fine, but you do then have to explain what you are observing. So if you run the following code:

In [10]:
def conjugate_gradient(A, b, x0):
    r0 = b - A @ x0
    p = r0
    x = x0
    for i in range(A.shape[0]):
        print(f"x_{i}: {x}")
        print(f"r_{i}: {r0}")
        Ap = A @ p
        alpha = np.dot(r0, r0) / np.dot(p, Ap)
        x = x + alpha * p
        r1 = r0 - alpha * Ap
        beta = np.dot(r1, r1)/np.dot(r0, r0)
        p = r1 + beta * p
        r0 = r1
    return x

In [11]:
conjugate_gradient(A, b, x0);

x_0: [0 0 0 0]
r_0: [1 1 0 0]
x_1: [0.33333333 0.33333333 0.         0.        ]
r_1: [ 0.          0.         -0.66666667 -0.66666667]
x_2: [ 0.5   0.5  -0.25 -0.25]
r_2: [0. 0. 0. 0.]
x_3: [nan nan nan nan]
r_3: [nan nan nan nan]


/tmp/ipykernel_147537/1102709967.py:9: RuntimeWarning: invalid value encountered in scalar divide
  alpha = np.dot(r0, r0) / np.dot(p, Ap)


you need to explain where the `nan`s are coming from: they arise in the calculation of $\alpha$ as $r$ and thus $p$ become zero, also the denominator becomes zero. Also you need to explain how you know that the exact solution is arrived at in the second iteration, namely by observing that the residual vector is exactly zero at that point.

If you use an implementation of the conjugate gradient algorithm with a stopping criterion, since we are asked for an exact solution and not an approximate answer within some tolerance, you need to at least check that the answer exactly satisfies the equation, i.e. that the residual is exactly zero. Note that even if you use something like `np.allclose` there is a default absolute tolerance `atol` (with a value of `1e-8`) that you are tolerating, so again you are not guaranteed to actually reach the exact solution. Of course in practice, in particular for larger and ill-conditioned systems, we may never be able to reach the solution exactly due to round off errors, but that's not an issue for the simple system here.

**2.2** What conclusion can you draw about the Krylov subspaces from the number of iterations that were required to reach the exact solution in the previous question? Work out the Krylov subspaces for this problem with the given initial guess ${\bf x}^0=(0,0,0,0)$ by giving a _linearly independent_ basis for each subspace.

## Solution <a class="tocSkip"></a>
As a general note, make sure you always answer each part/sub-question of every question. The first sub-question here gives you an immediate hint about the rest of the question: the conclusion you can draw from the fact that only two iterations are required in the Conjugate Gradient algorithm to reach the exact solution, is that
the solution ${\bf x}^{\ast}= {\bf x}^{(2)}$ must be in the second Krylov subspace $\mathcal{D}_1$:

$$
  {\bf x}^{\ast} - {\bf x}^{(0)} \in \mathcal{D}_1
$$

and that the Krylov subspace stops growing after iteration two, i.e. there only two Krylov subspaces, $\mathcal{D}_0$ and $\mathcal{D}_1$, and the last one has the maximum dimension of two, with all further subspaces being the same $\mathcal{D}_1 = \mathcal{D}_2 = \mathcal{D}_3$.

The first two subspaces can be calculated by computing the following vectors:

In [12]:
r0 = b - A @ x0
pprint(f"r0 = {r0}")
pprint(f"A @ r0 = {A @ r0}")

# just to confirm that these are indeed linearly dependent on the first two
pprint(f"A @ A @ r0 = {A @ A @r0}")
pprint(f"A @ A @ A @ r0 = {A @ A @ A @r0}")

'r0 = [1 1 0 0]'
'A @ r0 = [3 3 2 2]'
'A @ A @ r0 = [13 13 14 14]'
'A @ A @ A @ r0 = [67 67 82 82]'


Which means:
    
\begin{align}
  \mathcal{D}_0 = \operatorname{span}(\{{\bf r}^{(0)}\}) 
  &= \operatorname{span}(\{
  \begin{pmatrix} 1 \\ 1 \\ 0 \\ 0 \end{pmatrix}  
  \}) \\
  \mathcal{D}_1 = \operatorname{span}(\{{\bf r}^{(0)}, \underline{\bf A} {\bf r}^{(0)}\}) 
  &= \operatorname{span}(\{
  \begin{pmatrix} 1 \\ 1 \\ 0 \\ 0 \end{pmatrix},
  \begin{pmatrix} 3 \\ 3 \\ 2 \\ 2 \end{pmatrix},  
  \}) \\
  &= \operatorname{span}(\{
  \begin{pmatrix} 1 \\ 1 \\ 0 \\ 0 \end{pmatrix},
  \begin{pmatrix} 0 \\ 0 \\ 1 \\ 1 \end{pmatrix},  
  \})
\end{align}

and indeed the for third subspace $\mathcal{D}_2 = \operatorname{span}(\{{\bf r}^{(0)}, \underline{\bf A} {\bf r}^{(0)}, \underline{\bf A}^2 {\bf r}^{(0)}\})$ but as we saw $\underline{\bf A}^2 {\bf r}^{(0)}$ is a linear combination of ${\bf r}^{(0)}$ and $\underline{\bf A} {\bf r}^{(0)}$, and thus as expected $\mathcal{D}_2=\mathcal{D}_1$.  

**2.3** Use the Krylov subspaces to calculate the iteration vectors ${\bf x}^i$ that are generated by the GMRES algorithm. You do **not** need any GMRES implementation to answer this question; you can answer it purely from its definition as a Krylov subspace method.

## Solution <a class="tocSkip"></a>
Here you can use the fact that GMRES is a Krylov subspace method, and, when you realize that the definition of Krylov subspaces is independent of which particular Krylov subspace method you use, the Krylov subspaces derived in the previous question. Again, because we know that the exact solution is reachable in the second Krylov subspace, the GMRES algorithm will find the exact solution in the second iteration. So in fact you only need to work out what the solution vector is in the first iteration!

The GMRES method is defined as the Krylov subspace method that minimizes the residual, i.e. it chooses each new iteration to be in the point in the Krylov subspace where the residual $\|{\bf b} - \underline{\bf A}{\bf x}\|$ is minimal.

For the first iteration we have ${\bf x}^{(1)}-{\bf x}^{(0)}\in\mathcal{D}_0$ which means

$${\bf x}^{(1)} = \begin{pmatrix} \alpha \\ \alpha \\ 0 \\ 0 \end{pmatrix}$$

for some $\alpha$, which gives a residual of:

$$
{\bf r}^{(1)} = {\bf b} - \underline{\bf A}{\bf x}^{(1)}
=
\begin{pmatrix}
  1 \\ 1 \\ 0 \\ 0
\end{pmatrix}
-
\begin{pmatrix}
  2 & 1 & 1 & 1 \\
  1 & 2 & 1 & 1 \\
  1 & 1 & 3 & 1 \\
  1 & 1 & 1 & 3 \\
\end{pmatrix}
\begin{pmatrix}
  \alpha \\ \alpha \\ 0 \\ 0
\end{pmatrix}
=
\begin{pmatrix}
  1-3\alpha \\ 1-3\alpha \\ -2\alpha \\ -2\alpha
\end{pmatrix},
$$
the norm of which is
$$
\| {\bf r}^{(1)} \|^2 = 2 (1-3\alpha)^2 + 2(-2\alpha)^2 = 26\alpha^2 - 12\alpha + 2
$$
We find its minimum by solving
$$\frac{\partial \| {\bf r}^{(1)} \|^2}{\partial\alpha} = 52\alpha - 12 = 0
$$
which gives $\alpha=3/13$, and so
$$
{\bf x}^{(1)} = \begin{pmatrix} \frac 3{13} \\ \frac 3{13} \\ 0 \\ 0 \end{pmatrix}$$.

If you like to check your answer with scipy's gmres (this was not asked for of course!), you can use the following code:

In [164]:
import scipy.sparse.linalg as spl
pprint([3/13,3/13,0,0])
# need to use restart=1, otherwise the actual iterations that are performed
# is maxiter*restart
pprint(spl.gmres(A, b, maxiter=1, restart=1))

[0.23076923076923078, 0.23076923076923078, 0, 0]
(array([0.23076923, 0.23076923, 0.        , 0.        ]), 1)


In the second iteration you can immediately use the fact that exact solution ${\bf x}^\ast$ is in the second Krylov subspace: $\bf{x}^{(*)}-\bf{x}^{(0)}\in\mathcal{D}_1$ so that the minimum residual in that subspace is in fact achieved in ${\bf x}^\ast$. Thus

$$
{\bf x}^{(2)} = {\bf x}^{\ast} = 
\begin{pmatrix} \tfrac 12 \\ \tfrac 12 \\ -\tfrac 14 \\ -\tfrac 14 \end{pmatrix}$$

If you want to convince yourself of this fact, you could write

$${\bf x}^{(2)} = \begin{pmatrix} \alpha \\ \alpha \\ \beta \\ \beta \end{pmatrix}$$

using a linear combination of the basisvectors of $\mathcal{D}_1$ we worked out above. Then
$$
{\bf r}^{(2)} = {\bf b} - \underline{\bf A}{\bf x}^{(2)}
=
\begin{pmatrix}
  1 \\ 1 \\ 0 \\ 0
\end{pmatrix}
-
\begin{pmatrix}
  2 & 1 & 1 & 1 \\
  1 & 2 & 1 & 1 \\
  1 & 1 & 3 & 1 \\
  1 & 1 & 1 & 3 \\
\end{pmatrix}
\begin{pmatrix}
  \alpha \\ \alpha \\ \beta \\ \beta
\end{pmatrix}
=
\begin{pmatrix}
  1-3\alpha - 2\beta \\ 1-3\alpha - 2\beta \\ -2\alpha - 4\beta \\ -2\alpha - 4\beta
\end{pmatrix}$$
with norm-squared
$$
\| {\bf r}^{(2)} \|^2 = 2 (1-3\alpha - 2\beta)^2 + 2(-2\alpha - 4\beta)^2
$$
which we can minimize to get the same answer...

Again, we can check this as well by running scipy's gmres algorithm for exactly two iterations:

In [167]:
import scipy.sparse.linalg as spl
pprint([1/2, 1/2, -1/4, -1/4])
# performs restart*maxiter iterations
pprint(spl.gmres(A, b, maxiter=1, restart=2))

[0.5, 0.5, -0.25, -0.25]
(array([ 0.5 ,  0.5 , -0.25, -0.25]), 0)
